In [1]:
pip install seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 2.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 35.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.4/242.4 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 58.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 58.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 14.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install scipy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 61.9 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [10]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr

In [11]:
def load_data(db_name, query):
    conn = sqlite3.connect(db_name)
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

In [13]:
# データベースのテーブルとスキーマを表示する関数
def print_db_schema(db_name):
    # データベースに接続
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    # 利用可能なテーブルのリストを取得
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    print(f"データベース '{db_name}' のテーブル:")
    for table in tables:
        table_name = table[0]
        print(f"\nテーブル: {table_name}")

        # 各テーブルのスキーマ（列の構造）を取得
        cursor.execute(f"PRAGMA table_info({table_name});")
        columns = cursor.fetchall()
        for col in columns:
            print(f"  列: {col[1]}, 型: {col[2]}")

    # 接続を閉じる
    conn.close()

# 'weather_data.db' のスキーマを表示
print_db_schema('weather_data.db')

# 'mydatabase.sqlite' のスキーマを表示
print_db_schema('mydatabase.sqlite')

データベース 'weather_data.db' のテーブル:

テーブル: weather
  列: date, 型: TEXT
  列: pressure, 型: TEXT
  列: rainfall, 型: TEXT
  列: temp_avg, 型: TEXT
  列: temp_high, 型: TEXT
  列: temp_low, 型: TEXT
  列: vapor_pressure, 型: TEXT
  列: humidity, 型: TEXT
データベース 'mydatabase.sqlite' のテーブル:

テーブル: health_data
  列: date, 型: INTEGER
  列: temperature, 型: REAL
  列: wake_up_time, 型: INTEGER
  列: sleep_duration, 型: INTEGER
  列: headache, 型: INTEGER
  列: stomachache, 型: INTEGER
  列: sore_throat, 型: INTEGER
  列: nausea, 型: INTEGER
  列: runny_nose, 型: INTEGER
  列: cough, 型: INTEGER


In [17]:
import sqlite3
import pandas as pd
from scipy.stats import pearsonr

# データベースからデータを読み込む関数
def load_data(db_name, query):
    conn = sqlite3.connect(db_name)
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

# weather_data.db からデータを読み込む
weather_data = load_data('weather_data.db', 'SELECT * FROM weather')

# mydatabase.sqlite からデータを読み込む
health_data = load_data('mydatabase.sqlite', 'SELECT * FROM health_data')

# weather_data の日付のデータ型を変換（例：2023年1月を使用）
weather_data['date'] = pd.to_datetime('2023-01-' + weather_data['date'].astype(str))

# health_data の日付のデータ型を変換（例：2023年1月を使用）
health_data['date'] = pd.to_datetime('2023-01-' + health_data['date'].astype(str))

# データを日付で結合
combined_data = pd.merge(weather_data, health_data, on='date')

# 必要に応じてデータ型を変換（例：数値データの型など）
combined_data['temp_avg'] = pd.to_numeric(combined_data['temp_avg'], errors='coerce')
combined_data['temperature'] = pd.to_numeric(combined_data['temperature'], errors='coerce')

# 相関分析（例：平均気温と体温の相関）
correlation, p_value = pearsonr(combined_data['temp_avg'].dropna(), combined_data['temperature'].dropna())

print(f'相関係数: {correlation}, P値: {p_value}')

# 他の変数間の関連性も同様に分析可能


相関係数: -0.5634697027811427, P値: 0.011993854859702746
